### Análisis exploratorio

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("cybersecurity_attacks.csv")
pd.options.display.max_columns = None

In [3]:
# valoramos la información del dataset como datos no nulos y tipo de datos.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Timestamp               40000 non-null  object 
 1   Source IP Address       40000 non-null  object 
 2   Destination IP Address  40000 non-null  object 
 3   Source Port             40000 non-null  int64  
 4   Destination Port        40000 non-null  int64  
 5   Protocol                40000 non-null  object 
 6   Packet Length           40000 non-null  int64  
 7   Packet Type             40000 non-null  object 
 8   Traffic Type            40000 non-null  object 
 9   Payload Data            40000 non-null  object 
 10  Malware Indicators      20000 non-null  object 
 11  Anomaly Scores          40000 non-null  float64
 12  Alerts/Warnings         19933 non-null  object 
 13  Attack Type             40000 non-null  object 
 14  Attack Signature        40000 non-null

In [4]:
# identificamos las columnas con datos nulos.
df.isnull().any()

Timestamp                 False
Source IP Address         False
Destination IP Address    False
Source Port               False
Destination Port          False
Protocol                  False
Packet Length             False
Packet Type               False
Traffic Type              False
Payload Data              False
Malware Indicators         True
Anomaly Scores            False
Alerts/Warnings            True
Attack Type               False
Attack Signature          False
Action Taken              False
Severity Level            False
User Information          False
Device Information        False
Network Segment           False
Geo-location Data         False
Proxy Information          True
Firewall Logs              True
IDS/IPS Alerts             True
Log Source                False
dtype: bool

In [5]:
# analizamos con más detalle el dataset con la descripción de cada columna.
df.describe(include='all').head()

,Timestamp,Source IP Address,Destination IP Address,Source Port,Destination Port,Protocol,Packet Length,Packet Type,Traffic Type,Payload Data,Malware Indicators,Anomaly Scores,Alerts/Warnings,Attack Type,Attack Signature,Action Taken,Severity Level,User Information,Device Information,Network Segment,Geo-location Data,Proxy Information,Firewall Logs,IDS/IPS Alerts,Log Source
count,40000,40000,40000,40000.00000,40000.00000,40000,40000.000000,40000,40000,40000,20000,40000.000000,19933,40000,40000,40000,40000,40000,40000,40000,40000,20149,20039,19950,40000
unique,39997,40000,40000,NaN,NaN,3,NaN,2,3,40000,1,NaN,1,3,2,3,3,32389,32104,3,8723,20148,1,1,2
top,2022-06-11 14:28:15,103.216.15.12,84.9.164.252,NaN,NaN,ICMP,NaN,Control,DNS,Qui natus odio asperiores nam. Optio nobis ius...,IoC Detected,NaN,Alert Triggered,DDoS,Known Pattern A,Blocked,Medium,Ishaan Chaudhari,Mozilla/5.0 (compatible; MSIE 6.0; Windows NT ...,Segment C,"Ghaziabad, Meghalaya",39.123.165.122,Log Data,Alert Data,Firewall
freq,2,1,1,NaN,NaN,13429,NaN,20237,13376,1,20000,NaN,19933,13428,20076,13529,13435,6,35,13408,16,2,20039,19950,20116
mean,NaN,NaN,NaN,32970.35645,33150.86865,NaN,781.452725,NaN,NaN,NaN,NaN,50.113473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# descartamos las columnas con la condición "count=unique", ya que considero no aportarán información en el aprendizaje de los modelos.
df = df.drop(['Source IP Address','Destination IP Address','Payload Data'], axis=1)

In [7]:
# revisaremos las columnas con valores nulos para valorar si aportarán al entrenamiuento del modelo y transformarla de ser necesario.
df.loc[:, df.isnull().any()]

,Malware Indicators,Alerts/Warnings,Proxy Information,Firewall Logs,IDS/IPS Alerts
0,IoC Detected,NaN,150.9.97.135,Log Data,NaN
1,IoC Detected,NaN,NaN,Log Data,NaN
2,IoC Detected,Alert Triggered,114.133.48.179,Log Data,Alert Data
3,NaN,Alert Triggered,NaN,NaN,Alert Data
4,NaN,Alert Triggered,149.6.110.119,NaN,Alert Data
...,...,...,...,...,...
39995,IoC Detected,NaN,NaN,Log Data,Alert Data
39996,IoC Detected,NaN,60.51.30.46,Log Data,NaN
39997,IoC Detected,NaN,NaN,Log Data,Alert Data
39998,IoC Detected,Alert Triggered,137.76.130.8,Log Data,NaN


In [8]:
# descartaremos las columnas 'Alerts/Warnings, Proxy Information, IDS/IPS Alerts', ya que considero no aportarán información en el aprendizaje de los modelos.
df = df.drop(['Alerts/Warnings','Proxy Information','IDS/IPS Alerts'], axis=1)

In [9]:
# definimos valores a los campos NaN de las columnas 'Malware Indicators, Firewall Logs'
df['Malware Indicators'] = df['Firewall Logs'].fillna('IoC No Detected')
df['Firewall Logs'] = df['Firewall Logs'].fillna('No Log Data')

In [10]:
df['Timestamp'].sort_values()

17494    2020-01-01 00:43:27
33427    2020-01-01 01:14:07
26028    2020-01-01 03:02:42
38419    2020-01-01 03:11:48
29339    2020-01-01 03:29:20
                ...         
4778     2023-10-11 12:29:23
13787    2023-10-11 12:36:21
29107    2023-10-11 17:44:41
27995    2023-10-11 18:50:41
11677    2023-10-11 19:34:23
Name: Timestamp, Length: 40000, dtype: object

In [11]:
# transformaremos el dato de la columna 'Timestamp' a trimestres.
from datetime import datetime

df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Trimestre'] = df['Timestamp'].dt.quarter
df = df.drop(['Timestamp'], axis=1)

In [12]:
df['Trimestre'].value_counts()

Trimestre
3    10720
2    10625
1    10288
4     8367
Name: count, dtype: int64

In [13]:
# solamente trabajeremos con la información del sistema operativo de la 'Device Information'.
df['Device Information'] = df['Device Information'].str.split('(').str[0]
df['Device Information'] = df['Device Information'].str.strip()

In [14]:
df[['Device Information']].value_counts()

Device Information
Mozilla/5.0           31951
Opera/9.37.              67
Opera/9.89.              63
Opera/8.37.              59
Opera/9.75.              59
                      ...  
Opera/9.14.              32
Opera/9.21.              32
Opera/9.66.              31
Opera/9.82.              31
Opera/9.43.              30
Name: count, Length: 181, dtype: int64

In [15]:
# discriminamos la localidad y Región la columna 'Geo-location Data'.
df[['Localidad','Region']] = df['Geo-location Data'].str.split(',', expand=True)
df['Localidad'] = df['Localidad'].str.strip()
df['Region'] = df['Region'].str.strip()
df = df.drop(['Geo-location Data'], axis=1)

In [16]:
df[['Localidad','Region']]

,Localidad,Region
0,Jamshedpur,Sikkim
1,Bilaspur,Nagaland
2,Bokaro,Rajasthan
3,Jaunpur,Rajasthan
4,Anantapur,Tripura
...,...,...
39995,Nashik,Manipur
39996,Vadodara,Mizoram
39997,Mahbubnagar,Himachal Pradesh
39998,Rourkela,Arunachal Pradesh


In [17]:
df

,Source Port,Destination Port,Protocol,Packet Length,Packet Type,Traffic Type,Malware Indicators,Anomaly Scores,Attack Type,Attack Signature,Action Taken,Severity Level,User Information,Device Information,Network Segment,Firewall Logs,Log Source,Trimestre,Localidad,Region
0,31225,17616,ICMP,503,Data,HTTP,Log Data,28.67,Malware,Known Pattern B,Logged,Low,Reyansh Dugal,Mozilla/5.0,Segment A,Log Data,Server,2,Jamshedpur,Sikkim
1,17245,48166,ICMP,1174,Data,HTTP,Log Data,51.50,Malware,Known Pattern A,Blocked,Low,Sumer Rana,Mozilla/5.0,Segment B,Log Data,Firewall,3,Bilaspur,Nagaland
2,16811,53600,UDP,306,Control,HTTP,Log Data,87.42,DDoS,Known Pattern B,Ignored,Low,Himmat Karpe,Mozilla/5.0,Segment C,Log Data,Firewall,4,Bokaro,Rajasthan
3,20018,32534,UDP,385,Data,HTTP,IoC No Detected,15.79,Malware,Known Pattern B,Blocked,Medium,Fateh Kibe,Mozilla/5.0,Segment B,No Log Data,Firewall,3,Jaunpur,Rajasthan
4,6131,26646,TCP,1462,Data,DNS,IoC No Detected,0.52,DDoS,Known Pattern B,Blocked,Low,Dhanush Chad,Mozilla/5.0,Segment C,No Log Data,Firewall,3,Anantapur,Tripura
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,31005,6764,UDP,1428,Control,HTTP,Log Data,39.28,DDoS,Known Pattern A,Logged,Medium,Adira Madan,Mozilla/5.0,Segment A,Log Data,Firewall,2,Nashik,Manipur
39996,2553,28091,UDP,1184,Control,HTTP,Log Data,27.25,DDoS,Known Pattern A,Logged,High,Rati Dara,Mozilla/5.0,Segment C,Log Data,Firewall,1,Vadodara,Mizoram
39997,22505,25152,UDP,1043,Data,DNS,Log Data,31.01,DDoS,Known Pattern B,Blocked,Low,Samiha Joshi,Mozilla/5.0,Segment C,Log Data,Server,1,Mahbubnagar,Himachal Pradesh
39998,20013,2703,UDP,483,Data,FTP,Log Data,97.85,Malware,Known Pattern B,Ignored,Low,Rasha Chauhan,Mozilla/5.0,Segment B,Log Data,Server,3,Rourkela,Arunachal Pradesh


In [18]:
# guardamos el dataset con las transformaciones realizadas.

df.to_csv('./02_clean_cybersecurity_attacks.csv', sep=',', index=False)

In [19]:
# para nuestro primer análisis codificaremos todos los campos con 'LabelEncoder'.
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
columns_to_encode = [
    'Source Port', 'Destination Port', 'Protocol', 'Packet Length', 'Packet Type',
    'Traffic Type', 'Malware Indicators', 'Anomaly Scores', 'Attack Type',
    'Attack Signature', 'Action Taken', 'Severity Level', 'User Information',
    'Device Information', 'Network Segment', 'Firewall Logs', 'Log Source',
    'Trimestre', 'Localidad', 'Region'
]

for column in columns_to_encode:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

In [20]:
df

,Source Port,Destination Port,Protocol,Packet Length,Packet Type,Traffic Type,Malware Indicators,Anomaly Scores,Attack Type,Attack Signature,Action Taken,Severity Level,User Information,Device Information,Network Segment,Firewall Logs,Log Source,Trimestre,Localidad,Region
0,14137,7709,0,439,1,2,1,2827,2,1,2,1,22830,0,0,0,1,1,134,21
1,7587,21888,0,1110,1,2,1,5070,2,0,0,1,27434,0,1,0,0,2,60,17
2,7388,24463,2,242,0,2,1,8595,0,1,1,1,10139,0,2,0,0,3,61,20
3,8913,14681,2,321,1,2,0,1556,2,1,0,2,9216,0,1,1,0,2,135,20
4,2365,11942,1,1398,1,0,0,50,0,1,0,1,6462,0,2,1,0,2,20,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,14033,2665,2,1364,0,2,1,3870,0,0,2,2,1322,0,0,0,0,1,210,14
39996,731,12634,2,1120,0,2,1,2687,0,0,2,0,22352,0,2,0,0,0,307,16
39997,10087,11250,2,979,1,0,1,3058,0,1,0,1,25221,0,2,0,1,0,178,8
39998,8909,774,2,419,1,1,1,9613,2,1,1,1,22186,0,1,0,1,2,252,1


In [21]:
action_classes = label_encoders['Action Taken'].classes_
severity_classes = label_encoders['Severity Level'].classes_
attack_classes = label_encoders['Attack Type'].classes_

print("Categorías asignadas para Action Taken ['0' '1' '2']':", action_classes)
print("Categorías asignadas para Severity Level ['0' '1' '2']':", severity_classes)
print("Categorías asignadas para Attack Type ['0' '1' '2']':", attack_classes)


Categorías asignadas para Action Taken ['0' '1' '2']': ['Blocked' 'Ignored' 'Logged']
Categorías asignadas para Severity Level ['0' '1' '2']': ['High' 'Low' 'Medium']
Categorías asignadas para Attack Type ['0' '1' '2']': ['DDoS' 'Intrusion' 'Malware']


In [22]:
# guardaremos el dataset codificado para los siguientes análisis cuando cambiemos el objetivo a predecir.

df.to_csv('./03_codif_cybersecurity_attacks.csv', sep=',', index=False)